In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from langchain_teddynote import logging

logging.langsmith("JSON")

LangSmith 추적을 시작합니다.
[프로젝트명]
JSON


### 모듈 import

In [2]:
import pandas as pd
from langchain.text_splitter import CharacterTextSplitter
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain_community.vectorstores import FAISS
from langchain_upstage import UpstageEmbeddings, ChatUpstage
from langchain_anthropic import ChatAnthropic
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda 
from langchain.memory import ConversationBufferMemory
from langchain.schema import Document
from datetime import datetime
from langchain_community.document_loaders import JSONLoader
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_anthropic import ChatAnthropic
from langchain.schema.output_parser import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain.schema import BaseOutputParser
import json

## 1. 대분류 :의도파악하기

(가격)  
- Upstage Solar 1 mini :  0.25(preview동안 0원)    
  주문일때 키워드 뽑는건 잘함.   
  1초 미만(0~0.7초)  
  {'분류': '기타', '키워드': []}, {'분류': '추천', '키워드': []} 등 주문 아닌데도 키워드 항목 만들고 빈 리스트,,,  
  "처음부터 주문 다시할게" -> 취소(O) but 주문으로 판단  
    
- gpt-4o-mini-2024-07-18 : input 0.15, output 0.6  
- claude-3-5-sonnet-20240620 : input 3, output 15  
  가격차이가 꽤 많이 나는데, 4o와 claude 모델간 정확도가 큰 차이는 없어서(claude 더 높음) gpt4o 선택. 추후 도저히 안되면 claude로 변경
    

- GPT-4o 2024-08-06 : input 2.5, output 10

In [3]:
# chatGPT 4o mini

gpt4o = ChatOpenAI(model_name="gpt-4o-mini-2024-07-18", temperature=0.3)  #temperature 바꿔서도 실행해보기
# 1초대 속도, 근데 딱 하나가 최대시간(3초)차지
# claude와 정확도면에서 크게 차이나진 않음 -> 우선선택!


In [ ]:
# Claude 3.5 sonnet
claude = ChatAnthropic(model_name="claude-3-5-sonnet-20240620")
# 2초씩 걸린다... 정확도는 높음

In [8]:
intent_chain = PromptTemplate.from_template("""
아래 질문을 보고 사용자가 원하는 의도를 '주문', '추천', '취소', '결제', '기타' 중 하나로 정확하게 분류하세요.

분류 기준:
- 주문: 특정 메뉴를 정확히 주문하려는 경우 (예: '빅맥 하나 주세요', '감자튀김 추가')
- 추천: 특정 메뉴 이름 대신 추천을 요청하거나 메뉴 선택에 도움을 원하는 경우 (예: '뭐가 맛있나요?', '매콤한 음식 추천해줘')
- 취소: 이전에 진행된 주문을 취소하려는 경우 (예: '주문 취소해 주세요', '아까 주문한 것 취소하고 싶어요')
- 결제: 주문 완료 후 결제를 요청하는 경우 (예: '결제할게요', '주문 끝났어', '주문완료')
- 기타: 위의 네 가지 분류에 해당하지 않는 경우 (예: '화장실은 어디인가요?', '영업 시간은 언제인가요?')

<질문>
{question}
</질문>

질문의 분류와 해당 질문에 포함된 주요 키워드를 JSON 형식으로 출력하세요.

조건:
- 질문의 분류와 해당 질문에 포함된 주요 키워드를 딕셔너리 형식으로 출력하세요.
- 분류가 '주문'일 때만 키워드를 추출하세요. 다른 분류에서는 키워드를 출력하지 마세요
- 주문이 세트메뉴인 경우, 버거명에 세트메뉴 키워드를 포함하세요

예시 출력: {{"분류": "주문", "키워드": "빅맥"}}
예시 출력: {{"분류": "추천"}}
""") | gpt4o | StrOutputParser()

In [9]:
# intent 결과인 문자열을 딕셔너리로 변환(보이기엔 딕셔너리 형태여도 type:str으로 출력됨)
def parse_response(response_str):
    try:
        response_dict = json.loads(response_str)
    except json.JSONDecodeError:
        response_dict = {"error": "파싱 오류", "원본": response_str}
    return response_dict

In [11]:
intent_chain.invoke("완료할게")

'{"분류": "결제"}'

## task별 구분

주문(빠른주문) : 명확한 이름을 아는 경우 JSON 형식으로 출력. 명확한 이름을 모르는 경우 LLM으로 한번 걸러서 주문 진행.  
결제, 취소는 코딩으로 해결  
기타 : LLM 무한가동시켜서 의도분류 task중 하나 선택하도록 유도  

## 2. 빠른주문

In [16]:
category = intent.get('분류')
intent_keyword = intent.get('키워드')
print(category, type(category))
print(intent_keyword, type(intent_keyword))


주문 <class 'str'>
['불고기버거'] <class 'list'>


### 전체코드

필요작업   
LLM에서 추측한 메뉴가 여러개일 때, 둘 중 맞는게 있는지 확인  
맨 뒤에 의도분류 연결  
GPT, claude 모델 비교  
GPT temperature 비교


가격구성....헬게이트.....  
SHOPPING CART [[{}]] 최종 주문LIST에 또 APPEND하기  
'세트'이면 세트의 갯수를 물어보도록 하기  
세트, 단품 나눈데서 아예 메뉴 추가하면 되려나?  
버거가 세트메뉴이면 후렌치 후라이, 콜라 하나씩 엮어서 세트가격 묶고, 추가메뉴는 추가가격 계산하고, 세트구성이 완전히 되지 않으면 정정하는 작업을 '결제'모듈에서 실행  

#### 메뉴명 추출

In [18]:
#메뉴명 추출

file_path = '/home/yoojin/ML/aiffel/HackaThon/modu_hackaton/LLM/files/menu_1017.json'

with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# 메뉴명을 저장할 리스트
menu_list = []
for item in data:
    menu_name = item['page_content']['name']
    menu_list.append(menu_name)

print(menu_list)

['맥크리스피 스리라차 마요', '베토디 스리라차 마요', '더블 쿼터파운더 치즈', '쿼터파운더 치즈', '맥스파이시 상하이 버거', '토마토 치즈 비프 버거', '맥크리스피 디럭스 버거', '맥크리스피 클래식 버거', '빅맥', '트리플 치즈버거', '1955 버거', '맥치킨 모짜렐라', '맥치킨', '더블 불고기 버거', '불고기 버거', '슈슈 버거', '슈비 버거', '베이컨 토마토 디럭스', '치즈버거', '더블 치즈버거', '햄버거', '소시지 스낵랩', '맥윙 2조각', '맥윙 4조각', '맥윙 8조각', '상하이 치킨 스낵랩', '골든 모짜렐라 치즈스틱 4조각', '골든 모짜렐라 치즈스틱 2조각', '맥너겟 6조각', '맥너겟 4조각', '맥스파이시 치킨텐더 2조각', '후렌치 후라이', '스위트 앤 사워 소스', '스위트 칠리 소스', '케이준 소스', '그리머스 쉐이크', '자두 천도복숭아 칠러', '제주 한라봉 칠러', '코카콜라', '코카콜라 제로', '스프라이트', '환타', '바닐라 라떼', '카페라떼', '카푸치노', '아메리카노', '드립 커피', '아이스 바닐라 라떼', '아이스 카페라떼', '아이스 아메리카노', '아이스 드립 커피', '디카페인 바닐라 라떼', '디카페인 카페라떼', '디카페인 카푸치노', '디카페인 아메리카노', '디카페인 아이스 바닐라 라떼', '디카페인 아이스 카페라떼', '디카페인 아이스 아메리카노', '바닐라 쉐이크', '딸기 쉐이크', '초코 쉐이크', '생수', '애플 파이', '오레오 맥플러리', '베리 스트로베리 맥플러리']


#### 이름추측 LLM model

In [ ]:
def save_menu(self, keyword):
    for item in data:
        menu_name_normalized = re.sub(r'\s+', '', item['page_content']['name']).replace("버거", "").strip()
        keyword_normalized = re.sub(r'\s+', '', keyword).replace("버거", "").strip()
        if menu_name_normalized == keyword_normalized:
            return {
                "category": item['page_content']['category'],
                "name": item['page_content']['name'],
                "product_status": item['page_content']['product_status'],
                "description": item['page_content']['description'],
                "price": item['page_content']['price'],
                "set_burger_price": item['page_content']['set_burger_price'],
                "nutrition": item['page_content']['nutrition'],
                "origin_info": item['page_content']['origin_info'],
                "set_menu": False
            }
    return None

In [ ]:
# 커스텀 출력 파서 정의
class CustomOutputParser(BaseOutputParser):
    def parse(self, text: str) -> dict:
        # 항상 'answer' 키로 '정답'을 반환
        return {"answer": text.strip()}

# llm = ChatOpenAI(model_name="gpt-4o-mini-2024-07-18", temperature=0.3)
llm = ChatAnthropic(model_name="claude-3-5-sonnet-20240620")
output_parser = CustomOutputParser()

# 프롬프트 템플릿 정의
name_chain = PromptTemplate.from_template("""
사용자의 질문과 메뉴 리스트를 비교하여 사용자의 질문과 비슷한 메뉴가 있는지 확인하세요.
비슷한 이름의 메뉴가 있다면 비슷한 메뉴를 모두 출력하세요
비슷한 메뉴가 없다면 "없음"이라고 출력하세요.
출력 시 메뉴 이름만 출력하세요

<메뉴 리스트>
['맥크리스피 스리라차 마요', '베토디 스리라차 마요', '더블 쿼터파운더 치즈', '쿼터파운더 치즈', '맥스파이시 상하이 버거', '토마토 치즈 비프 버거', '맥크리스피 디럭스 버거', '맥크리스피 클래식 버거', '빅맥', '트리플 치즈버거', '1955 버거', '맥치킨 모짜렐라', '맥치킨', '더블 불고기 버거', '불고기 버거', '슈슈 버거', '슈비 버거', '베이컨 토마토 디럭스', '치즈버거', '더블 치즈버거', '햄버거', '소시지 스낵랩', '맥윙 2조각', '맥윙 4조각', '맥윙 8조각', '상하이 치킨 스낵랩', '골든 모짜렐라 치즈스틱 4조각', '골든 모짜렐라 치즈스틱 2조각', '맥너겟 6조각', '맥너겟 4조각', '맥스파이시 치킨텐더 2조각', '후렌치 후라이', '스위트 앤 사워 소스', '스위트 칠리 소스', '케이준 소스', '그리머스 쉐이크', '자두 천도복숭아 칠러', '제주 한라봉 칠러', '코카콜라', '코카콜라 제로', '스프라이트', '환타', '바닐라 라떼', '카페라떼', '카푸치노', '아메리카노', '드립 커피', '아이스 바닐라 라떼', '아이스 카페라떼', '아이스 아메리카노', '아이스 드립 커피', '디카페인 바닐라 라떼', '디카페인 카페라떼', '디카페인 카푸치노', '디카페인 아메리카노', '디카페인 아이스 바닐라 라떼', '디카페인 아이스 카페라떼', '디카페인 아이스 아메리카노', '바닐라 쉐이크', '딸기 쉐이크', '초코 쉐이크', '생수', '애플 파이', '오레오 맥플러리', '베리 스트로베리 맥플러리']
</메뉴 리스트>

<질문>
{question}
</질문>

""") | llm | output_parser



#### 빠른주문 모듈

keyword가 없거나, menu list에 없으면 추천시스템으로 넘어가기

In [ ]:
intent_result = intent_chain.invoke(question)
category = intent_result.get('분류')
intent_keyword = intent_result.get('키워드')
print(f"category: {category}, intent_keyword: {intent_keyword}")

unrecognized_keywords = []
recommend = False
menu_data = []
shopping_cart = ShoppingCart()

if category == '주문':
    if intent_keyword:
        for keyword in intent_keyword: 
            print(f"keyword: {keyword}")
            menu_data_json = save_menu(keyword)
            
            if menu_data_json is None:
                # 이름 추측 체인을 사용해 비슷한 메뉴 찾기
                ### 지금은 비슷한 메뉴 하나만 고려하는데, 여러개 중 어떤건지 되묻는 기능도 있으면 좋을듯
                similar_menu = name_chain.invoke(keyword)["answer"]
                print(f"similar_menu: {similar_menu}")
                
                if similar_menu != "없음":
                    menu_data_json = save_menu(similar_menu)

            if menu_data_json is None:
                unrecognized_keywords.append(keyword)
                recommend = True
            else:
                print(f"menu_data_json: {menu_data_json}")
                print(f"type: {type(menu_data_json)}")
                if "세트" in keyword:
                    menu_data_json['set_menu'] = True
                menu_data.append(menu_data_json)

        for order_data in menu_data:
            ###### 수량.... 얘도 LLM 연결해서 숫자만 추출해야할것같은데...
            ### 세트는 한번에 물어보도록 해야할 것 같음... 어떻게...
            num = int(input(f"{order_data['name']} 메뉴가 맞으신가요? 수량은 몇 개 드릴까요? "))
            shopping_cart.add_to_cart(order_data, num)

        if unrecognized_keywords:
            print(f"선택하신 메뉴 중 확인할 수 없는 항목이 있습니다: {unrecognized_keywords}. 추천 챗봇을 연결해 드리겠습니다.")
            recommend = True
        print(f"shopping_cart: {shopping_cart}")
    else:
        print("주문 키워드가 없습니다. 추천시스템으로 연결해드리겠습니다.")
        recommend = True
else:
    print("추천시스템으로 연결해드리겠습니다.")
    recommend = True

### 장바구니 넣기(장바구니 모듈 python 만들기)

단품, 세트 각각 딕셔너리에 묶어서 [[{단품1}], [{세트버거1},{세트사이드1},{세트음료1}],[{단품2}]]로 구성


In [ ]:
class ShoppingCart:
    def __init__(self):
        self.cart = []

    def add_to_cart(self, order_data, num):
        cart = {}
        cart["name"] = order_data["name"]
        cart["num"] = num
        cart["price"] = order_data["price"]
        self.cart.append(cart)

In [ ]:
cart = ShoppingCart()
cart.add_to_cart(menu_data_json, 1)

In [ ]:
print(shopping_cart)

## 3. 추천 시스템, memory & langgraph test

In [5]:
file_dir = "/home/yoojin/ML/aiffel/HackaThon/modu_hackaton/LLM/files/menu_1017.json"

with open(file_dir, 'r', encoding='utf-8') as file:
    content = file.read()
    # JSON 객체들을 리스트로 변환
    json_objects = json.loads(content)

# 파싱된 JSON 객체들을 문서로 변환
docs = [
    Document(
        page_content=json.dumps(obj['page_content'], ensure_ascii=False),
    )
    for obj in json_objects
]

file_name = file_dir.split("/")[-1]
cache_dir = LocalFileStore(f"./.cache/embeddings/{file_name}")

# CharacterTextSplitter를 사용하여 문서 분할
text_splitter = CharacterTextSplitter(separator="\n\n", chunk_size=100, chunk_overlap=0)
split_docs = text_splitter.split_documents(docs)

embeddings = UpstageEmbeddings(
    model="solar-embedding-1-large"
)

cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings=embeddings,
    document_embedding_cache=cache_dir,
    namespace="solar-embedding-1-large",
)
vectorstore = FAISS.from_documents(
    split_docs,
    cached_embedder,
)

faiss = vectorstore.as_retriever(search_kwargs={"k": 4})

bm25 = BM25Retriever.from_documents(split_docs)
bm25.k = 4

ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25, faiss],
    weights=[0.3, 0.7],
    search_type="mmr",
)

In [ ]:
# 메시지를 저장하는 리스트

def save_message(message, role):
    messages.append({"message": message, "role": role})
    
def get_memory():
    return ConversationBufferMemory(
        return_messages=True,
        memory_key="chat_history"
    )

In [6]:
llm = ChatAnthropic(model_name="claude-3-5-sonnet-20240620")
# memory = get_memory()

In [ ]:
# recommend_template = ChatPromptTemplate.from_messages([
#     ("system",
#     """
#     당신은 맥도날드 가게의 점원입니다. 
#     나이가 많은 노인 고객의 주문을 도와주세요.
#     나이와 관련된 어떤한 호칭도 하지 말고 '고객님'으로 부르세요.
#     어린아이도 이해하기 쉬운 단어로 설명해주세요.
#     상냥하고, 친절하고, 존댓말로 사용자의 질문에 답변을 해주세요. 
#     간결하게 답변해주세요.
    
#     반드시 제공된 정보만을 사용하여 질문에 대답하세요.
#     질문과 가장 관련성이 높은 정보를 찾아서 답변하세요.
#     메뉴추천은 최대 4개까지만 추천해주세요.
#     충분한 정보를 바탕으로 정확히 답변할 수 있는 경우에만 답변하세요.
#     항상 chat history를 고려하여 이전 대화내용과 관련되게 대답하세요

#     답변이 확실하지 않을 경우, "죄송합니다. 해당 질문에 대한 답을 찾을 수 없습니다."라고 답하세요.
    
#     주문 조건
#     - 원하는 메뉴가 없이 추천을 받고싶으면 신메뉴를 추천해주세요.
#     - 주문메뉴는 세트와 단품으로 구분한다.
#     - 세트메뉴는 버거, 사이드, 음료가 반드시 포함되어야 한다.
#     - 세트메뉴는 기본 미디엄 사이즈로 구성되며, 라지 사이즈로 변경 시 800원의 차액이 부과된다.
#     - 세트메뉴의 기본 구성 사이드는 후렌치 후라이 미디엄 1개, 음료는 코카콜라 미디엄 1개이다.
#     - 세트메뉴의 사이드는 코울슬로로 변경 가능하며 추가금액은 없다.
#     - 세트메뉴의 음료는 '음료'카테고리 내에서 변경 가능하며, 변경할 경우 해당 메뉴끼리의 차액이 추가되는 점을 안내해 드린다.
#     - 세트메뉴의 가격을 물어보면 'set_burger_price'의 값에 5600원을 더한 총 금액을 안내해 드린다.
#     - 단품 후렌치후라이는 스몰, 미디엄, 라지 중 하나를 선택할 수 있으며, 스몰 사이즈는 2300원, 미디움 사이즈는 3000원, 라지 사이즈는 4000원이다.
#     - 단품 음료는 미디엄, 라지 중 하나를 선택할 수 있으며, 미디엄 사이즈는 2600원, 라지 사이즈는 3100원이다.
#     - 추가 주문이 필요한지 묻고, 추가 주문이 없으면 최종 주문결과를 출력한다.
    
#     주문결과 출력 :
#     - 주문 결과는 세트메뉴와 단품메뉴를 각각 리스트로 묶어서 출력한다.
#     - 주문 결과는 json 형식으로 출력한다.

#     주문 결과 예시: 
#     [
#         [
#             {{'name': '불고기 버거', 'num': '1', 'price': '4200', 'set_menu': True, 'set_burger_price': '4000'}}, 
#             {{'name': '후렌치 후라이', 'num': '1', 'price': '3000'}},
#             {{'name': '코카콜라', 'num': '1', 'price': '2600'}}
#         ],
#         [
#             {{'name': '상하이 치킨 스낵랩', 'num': '1', 'price': '4000'}}
#         ]
#     ]
#     Context: {context}
#     Chat History: {chat_history}
#     """),
#     ("human", "{question}"),
# ])

In [ ]:
recommend_template = ChatPromptTemplate.from_messages([
    ("system",
    """
    당신은 맥도날드의 친절한 점원입니다.
    고객님의 주문을 도와드리세요. 나이와 상관없이 '고객님'이라고만 부르고, 모든 설명은 어린아이도 이해할 수 있게 간단하게 해주세요. 친절하고 상냥하게 존댓말로 응대합니다.
    
    대답할 때는 다음 사항을 기억하세요:
    1. 제공된 정보만으로 답변합니다.
    2. 질문과 가장 관련성이 높은 정보를 찾아서 답변하세요.
    3. 메뉴 추천은 4개 이하로 제한하고, 신메뉴를 먼저 추천하세요.
    4. 항상 대화 기록을 고려해 대답하세요.
    5. 확실하지 않은 경우 "죄송합니다. 해당 질문에 대한 답을 찾을 수 없습니다."라고 답하세요.

    **주문 조건:**
    - 메뉴는 세트와 단품으로 구분됩니다.
    - 세트메뉴는 버거, 사이드, 음료가 포함되며, 미디엄 사이즈가 기본입니다.
    - 사이즈 업그레이드는 800원 추가 요금이 부과됩니다.
    - 사이드는 후렌치 후라이 미디엄이 기본이며, 코울슬로로 무료 변경 가능합니다.
    - 음료는 코카콜라 미디엄이 기본이며, 음료 변경 시 차액이 발생할 수 있습니다.
    - 세트메뉴 가격은 'set_burger_price'에 5600원을 더한 금액입니다.
    - 단품 후렌치 후라이는 스몰(2300원), 미디엄(3000원), 라지(4000원) 중 선택할 수 있습니다.
    - 단품 음료는 미디엄(2600원) 또는 라지(3100원)입니다.
    
    **주문 절차:**
    - 먼저 고객의 주문을 받고 메뉴가 완성되면 주문이 완료되었는지 한번만 물어보세요.
    - 메뉴의 주문이 완성되지 않은 경우 주문완료가 되었는지 묻지 마세요.
    - 주문이 모두 완료되면 주문 결과를 출력하세요.
    
    **주문 결과 출력:**
    - 세트와 단품 메뉴를 각각 리스트로 구분해 JSON 형식으로 출력합니다.
    
    **주문 결과 예시:**
    [
        [
            {{'name': '불고기 버거', 'num': '1', 'price': '4200', 'set_menu': True, 'set_burger_price': '4000'}},
            {{'name': '후렌치 후라이', 'num': '1', 'price': '3000','set_menu': False}},
            {{'name': '코카콜라', 'num': '1', 'price': '2600','set_menu': False}}
        ],
        [
            {{'name': '상하이 치킨 스낵랩', 'num': '1', 'price': '4000','set_menu': False}}
        ]
    ]
    **Context:** {context}
    **Chat History:** {chat_history}
    """),
    ("human", "{question}"),
])

In [7]:
recommend_template = ChatPromptTemplate.from_messages([
    ("system",
    """
    당신은 맥도날드의 친절한 점원입니다.
    고객님의 주문을 도와드리세요. 나이와 상관없이 '고객님'이라고만 부르고, 모든 설명은 어린아이도 이해할 수 있게 간단하게 해주세요. 친절하고 상냥하게 존댓말로 응대합니다.
    
    대답할 때는 다음 사항을 기억하세요:
    1. 제공된 정보만으로 답변합니다.
    2. 질문과 가장 관련성이 높은 정보를 찾아서 답변하세요.
    3. 메뉴 추천은 4개 이하로 제한하고, 신메뉴를 먼저 추천하세요.
    4. 항상 대화 기록을 고려해 대답하세요.
    5. 확실하지 않은 경우 "죄송합니다. 해당 질문에 대한 답을 찾을 수 없습니다."라고 답하세요.
    6. 메뉴에 대한 설명을 제공할 때, 메뉴의 세부정보(이름)을 metadata에 넣고 사용자에게는 설명을 출력하세요.
    
    예시응답 : 
    {{
        "metadata": {{
            "menu_name": "맥크리스피 디럭스 버거",
        }},
        "content": "맥크리스피 디럭스 버거는 바삭한 치킨 패티와 상큼한 양상추, 토마토가 어우러진 버거입니다."
    }}

    **주문 조건:**
    - 메뉴는 세트와 단품으로 구분됩니다.
    - 세트메뉴는 버거, 사이드, 음료가 포함되며, 미디엄 사이즈가 기본입니다.
    - 사이즈 업그레이드는 800원 추가 요금이 부과됩니다.
    - 사이드는 후렌치 후라이 미디엄이 기본이며, 코울슬로로 무료 변경 가능합니다.
    - 음료는 코카콜라 미디엄이 기본이며, 음료 변경 시 차액이 발생할 수 있습니다.
    - 세트메뉴 가격은 'set_burger_price'에 5600원을 더한 금액입니다.
    - 단품 후렌치 후라이는 스몰(2300원), 미디엄(3000원), 라지(4000원) 중 선택할 수 있습니다.
    - 단품 음료는 미디엄(2600원) 또는 라지(3100원)입니다.
    
    **주문 절차:**
    - 먼저 고객의 주문을 받고 메뉴가 완성되면 주문이 완료되었는지 한번만 물어보세요.
    - 메뉴의 주문이 완성되지 않은 경우 주문완료가 되었는지 묻지 마세요.
    - 주문이 모두 완료되면 주문 결과를 출력하세요.
    
    **주문 결과 출력:**
    - 세트와 단품 메뉴를 각각 리스트로 구분해 JSON 형식으로 출력합니다.
    
    **주문 결과 예시:**
    [
        [
            {{'name': '불고기 버거', 'num': '1', 'price': '4200', 'set_menu': True, 'set_burger_price': '4000'}},
            {{'name': '후렌치 후라이', 'num': '1', 'price': '3000','set_menu': False}},
            {{'name': '코카콜라', 'num': '1', 'price': '2600','set_menu': False}}
        ],
        [
            {{'name': '상하이 치킨 스낵랩', 'num': '1', 'price': '4000','set_menu': False}}
        ]
    ]
    **Context:** {context}
    **Chat History:** {chat_history}
    """),
    ("human", "{question}"),
])

In [8]:
recommend_chain = {
    "context": ensemble_retriever,
    "chat_history": RunnableLambda(lambda _: self.memory.load_memory_variables({})["chat_history"]),
    "question": RunnablePassthrough()
} | recommend_template | self.model

NameError: name 'self' is not defined

In [ ]:
messages = []
memory.clear()

while True:
    # 사용자로부터 입력 받기
    user_message = input("질문을 입력해주세요: ").strip()

    # 입력이 비어있는 경우 반복문을 계속 진행
    if not user_message:
        continue
    
    save_message(user_message, "user")
    
    response = recommend_chain.invoke(user_message)
    ai_response = response.content if hasattr(response, 'content') else response
    save_message(ai_response, "assistant")
    print(f"AI : {ai_response}")
    
    memory.save_context({"input": user_message}, {"output": ai_response})

    # 주문 결과가 json 형식인지 확인하여 while문 종료
    try:
        # 문자열을 JSON으로 변환 시도
        parsed_response = json.loads(ai_response)
        
        # JSON 변환이 성공하면 프로그램 종료
        print("주문이 완료되었습니다. 프로그램을 종료합니다.")
        break
    except json.JSONDecodeError:
        # JSON 변환 실패 시, 반복 계속
        pass


In [ ]:
memory.load_memory_variables({})["chat_history"]

In [ ]:
print(messages)

In [16]:
import re
import json

def extract_json_data(text):
    try:
        # 중첩된 배열과 JSON 객체를 모두 추출하는 정규식
        json_match = re.search(r'\{.*\}', text, re.DOTALL)
        if json_match:
            # JSON 배열만 추출해서 문자열로 반환
            json_str = json_match.group(0)
            
            # 문자열을 Python 리스트/딕셔너리로 변환
            return json.loads(json_str)
        else:
            return None
    except json.JSONDecodeError as e:
        print(f"JSON Decode Error: {e}")
        return None
    except Exception as e:
        print(f"Error: {e}")
        return None

# 주어진 ai_response에서 JSON 데이터만 추출
ai_response =  '''{
    "completion": true,
    "reply": "네, 고객님. 주문이 완료되었습니다. 맥크리스피 스리라차 마요 버거 세트 준비해 드리겠습니다. 맛있게 드세요!",
    "order": [
        [
            {"name": "맥크리스피 스리라차 마요 버거", "num": "1", "price": "7500", "set_menu": true, "set_price": "8900"},
            {"name": "후렌치후라이", "num": "1", "price": "3000", "set_menu": true, "set_price": "0"},
            {"name": "코카콜라", "num": "1", "price": "2600", "set_menu": true, "set_price": "0"}
        ]
    ]
}'''

# JSON 데이터 추출
extracted_json = extract_json_data(ai_response)

# 추출된 데이터 출력
print(extracted_json)

# 추출된 데이터가 리스트와 딕셔너리로 인식되는지 확인
if isinstance(extracted_json, list):
    print("데이터는 리스트로 인식되었습니다.")
else:
    print("데이터가 리스트로 변환되지 않았습니다.")


{'completion': True, 'reply': '네, 고객님. 결제를 진행하겠습니다. 주문하신 빅맥 세트 미디엄에 음료 라지 사이즈 변경으로 총 8,900원입니다. 맛있게 드세요!', 'order': [[{'name': '빅맥', 'num': '1', 'price': '6300', 'set_menu': True, 'set_price': '8900'}, {'name': '후렌치후라이', 'num': '1', 'price': '3000', 'set_menu': True, 'set_price': '0'}, {'name': '코카콜라 라지', 'num': '1', 'price': '00', 'set_menu': True, 'set_price': '0'}]]}
데이터가 리스트로 변환되지 않았습니다.


In [17]:
print(type(extracted_json))

<class 'dict'>


In [18]:
print(extracted_json['completion'])
print(type(extracted_json['completion']))

True
<class 'bool'>


## 주문모듈 정리

In [2]:
class ShoppingCart():
    def __init__(self):
        self.cart = []

    def add_to_cart(self, menu,num):
        cart_item = {"name" : menu, "num" : num}
        self.cart.append(cart_item)
        return self.cart
    
shopping_cart = ShoppingCart()
cart = shopping_cart.cart
print(cart)

[]


In [10]:
bool(cart)

False

In [11]:
shopping_cart.add_to_cart('빅맥',2)
print(cart)
bool(cart)

[{'name': '빅맥', 'num': 2}]


True

In [3]:
def intent_module(intent):
    print('의도분류모델 실행')
    if intent =='주문':
        intent_keyword = '감튀'
    else:
        intent_keyword = None
    parsed_intent = {'category':{intent}, 'intent_keyword':[intent_keyword]}
    category = parsed_intent['category']
    keyword = parsed_intent['intent_keyword']
    return category, keyword

def fast_order_module(keyword, word):
    print("빠른주문모델 실행")
    success =  bool(word)
    if success :
        shopping_cart.add_to_cart("빠른주문메뉴",1)
        return success
    else :
        return success
        
    # True(주문성공)  : add_shopping_bag
    # False(주문실패) : 추천주문 모듈로 넘기기
    
def recommend_module(menu, num):
    print("추천주문모델 실행")
    # while문 안에서 대화 이어나가기
    # 모든 출력문은 json 형식
    # breakpoint(원래 json 인식이었는데...불가능, '결제'등 특정 키워드 출력하게)
    # parallel chain?
    # 장바구니에 내역 담고 끝내기
    shopping_cart.add_to_cart(menu, num)
    

In [4]:
while True:
    user_intent = input("의도")
    if not user_intent:  # 메세지 입력받을 때까지 대기
        print("대기중입니다")
        continue

    if not cart:  # 장바구니가 비어있다면
        intent_result = intent_module(user_intent)
        category, keyword = intent_result
        print(category, keyword)
        if category == '주문':
            fast_order_success = input("빠른주문 성공여부(True/False)")
            fast_order_success = bool(fast_order_success)
            fast_order_result = fast_order_module(keyword, fast_order_success)
            if not fast_order_result:  # 주문 실패했을 때 
                menu = input("주문성공 메뉴")
                num = int(input("주문메뉴 갯수"))
                recommend_module(menu, num)
                print(cart)
            else:  # 주문 성공했을 때
                print('빠른주문 성공')
                print(cart)
        elif category == '취소':
            print('취소 모듈 넘어가기')
            pass
        elif category == '결제':
            print('결제 모듈 넘어가기')
            pass
        else:
            menu = input("주문성공 메뉴")
            num = int(input("주문메뉴 갯수"))
            recommend_module(menu, num)
            print(cart)
    else:
        print('2nd cycle start')


의도분류모델 실행
{'주문'} ['감튀']


In [ ]:
{{
    "주문완료" : False,
    "LLM 응답" : "네, 고객님. 새우버거 메뉴를 알려드릴게요. 맥도날드에는 '슈슈 버거'와 '슈비 버거' 두 가지 새우버거가 있어요. '슈슈 버거'는 탱글한 통새우살이 가득 들어있는 버거예요. 단품은 5,500원, 세트는 7,300원이에요."
    "주문내역" : 
    [
        [
            {{'name': '불고기 버거', 'num': '1', 'price': '4200', 'set_menu': True, 'set_price': '4000'}},
            {{'name': '후렌치 후라이', 'num': '1', 'price': '3000','set_menu': False}},
            {{'name': '코카콜라', 'num': '1', 'price': '2600','set_menu': False}}
        ],
        [
            {{'name': '상하이 치킨 스낵랩', 'num': '1', 'price': '4000','set_menu': False}}
        ]
    ]
}}